In [1]:
import torch

from transformers import AutoConfig, AutoTokenizer, LlamaForCausalLM
from model import DiscreteDiffusionModel, get_anneal_attn_mask


/home/exx/miniconda3/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Use provided 4d attn-mask


In [2]:
id_model = 'diffusionfamily/diffullama'
device = 'cuda:0'

tokenizer = AutoTokenizer.from_pretrained(id_model)
config = AutoConfig.from_pretrained(id_model)
config.use_cache = False

model = LlamaForCausalLM.from_pretrained(
    id_model,
    config=config,
    torch_dtype=torch.bfloat16
).eval()


Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00,  8.91it/s]


In [ ]:
model = DiscreteDiffusionModel(
    model=model,
    config=config,
    tokenizer=tokenizer,
    device=device
).eval().to(device)

In [3]:
x = tokenizer.encode(
    "good morning my friend",
    padding="max_length",   # pad up to max_length
    truncation=True,        # truncate longer ones (recommended)
    max_length=32,         # the target length
    return_tensors='pt'
).to(device)

attention_mask = get_anneal_attn_mask(x.shape[1], x.shape[0], dtype=torch.bfloat16, device=x.device, attn_mask_ratio=1.0).to(device) # all 0

In [4]:
model(x, attention_mask)

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


jinyu: use_cache None self.config.use_cache True
jinyu: self.config: LlamaConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "diffusionfamily/diffullama",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 4096,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.49.0",
  "use_cache": true,
  "vocab_size": 32000
}

jinyuj: use_cache: True


IndexError: tuple index out of range